In [1]:
import polars as pl
import sys
import os

sys.path.append(os.path.dirname(os.path.abspath("")))

from libraries.client_stashapp import get_stashapp_client, StashAppClient

stash = get_stashapp_client()
stash_client = StashAppClient()

dUsing stash (v0.28.1-89-g642b0f22) endpoint at http://localhost:6969/graphql
dUsing stash (v0.28.1-89-g642b0f22) endpoint at http://localhost:6969/graphql


In [21]:
full_movie_tag = stash.find_tag("Full Movie")["id"]
compilation_tag = stash.find_tag("Compilation")["id"]
multiple_sex_scenes_in_a_scene_tag = stash.find_tag("Multiple Sex Scenes in a Scene")[
    "id"
]
behind_the_scenes_tag = stash.find_tag("Behind the Scenes")["id"]
tv_series_tag = stash.find_tag("TV Series")["id"]
non_sex_performer_tag = stash.find_tag("Non-Sex Performer")["id"]
virtual_sex_tag = stash.find_tag("Virtual Sex")["id"]
missing_performer_male_tag = stash.find_tag("Missing Performer (Male)")["id"]
missing_performer_female_tag = stash.find_tag("Missing Performer (Female)")["id"]

excluded_tags = [
    full_movie_tag,
    compilation_tag,
    multiple_sex_scenes_in_a_scene_tag,
    behind_the_scenes_tag,
    tv_series_tag,
    non_sex_performer_tag,
    virtual_sex_tag,
    missing_performer_male_tag,
    missing_performer_female_tag,
]

group_makeup_verified_tag_id = stash.find_tag("Group Makeup Verified")["id"]
group_makeup_calculated_parent_tag = stash.find_tag(
    "Group Makeup Calculated", fragment="id name children { id name }"
)
group_makeup_calculated_tag_ids = [
    tag["id"] for tag in group_makeup_calculated_parent_tag["children"]
]

group_makeup_generic_parent_tag = stash.find_tag(
    "Group Makeup Generic", fragment="id name children { id name }"
)
group_makeup_generic_parent_tag

group_makeup_generic_tags = [tag for tag in group_makeup_generic_parent_tag["children"]]
group_makeup_generic_tag_ids = [tag["id"] for tag in group_makeup_generic_tags]

group_makeup_specific_parent_tag = stash.find_tag(
    "Group Makeup Specific",
    fragment="id name children { id name children { id name } }",
)
# List tag ids for child tags of child tags of group makeup specific parent tag
group_makeup_specific_tags = [
    grandchild
    for child in group_makeup_specific_parent_tag["children"]
    for grandchild in child["children"]
]
group_makeup_specific_tag_ids = [tag["id"] for tag in group_makeup_specific_tags]

In [22]:
excluded_scenes_with_group_makeup_calculated = []
excluded_scenes = stash.find_scenes(
    {"tags": {"value": excluded_tags, "modifier": "INCLUDES"}},
    fragment="id title date tags { id name }",
)
for scene in excluded_scenes:
    scene_tag_ids = [tag["id"] for tag in scene["tags"]]
    if any(tag_id in group_makeup_calculated_tag_ids for tag_id in scene_tag_ids):
        matching_tag = next(
            (
                tag
                for tag in scene["tags"]
                if tag["id"] in group_makeup_calculated_tag_ids
            ),
            None,
        )
        if matching_tag:
            print(
                "Scene",
                scene["id"],
                "has group makeup calculated tag:",
                matching_tag["name"],
            )
            excluded_scenes_with_group_makeup_calculated.append(scene["id"])



Scene 6603 has group makeup calculated tag: Group Makeup Calculated: 7F4M
Scene 5196 has group makeup calculated tag: Group Makeup Calculated: 1F1M
Scene 10106 has group makeup calculated tag: Group Makeup Calculated: 1F5M
Scene 30072 has group makeup calculated tag: Group Makeup Calculated: 1F1M


In [4]:
stash_client.update_tags_for_scenes(
    excluded_scenes_with_group_makeup_calculated, [], ["Group Makeup Calculated: 1F"]
)


{'6603': True,
 '13782': True,
 '5196': True,
 '22512': True,
 '10106': True,
 '30072': True}

In [23]:
all_tags = pl.DataFrame(stash.find_tags(fragment="id name"))
all_tags

id,name
str,str
"""5045""","""2D Available"""
"""5049""","""3D Available"""
"""5050""","""3K Available"""
"""5051""","""3rd Person Narrative"""
"""5053""","""4:3 Aspect Ratio"""
…,…
"""7559""","""Young Man (22–30)"""
"""7560""","""Young Woman (22–30)"""
"""7563""","""Zentai"""


In [25]:
class GroupMakeupMapping:
    def __init__(self, all_tags_df):
        self.mappings = (
            {}
        )  # calculated_tag_name -> (generic_tag_name, specific_tag_name)
        self.ignored_tags = set()  # Set of calculated tags to ignore
        self.valid_tag_names = set(all_tags_df["name"])

    def add_mapping(self, calculated_name, generic_name, specific_name=None):
        # Strip the prefix if present
        if calculated_name.startswith("Group Makeup Calculated: "):
            calculated_name = calculated_name[len("Group Makeup Calculated: ") :]

        # Validate generic tag exists
        if generic_name not in self.valid_tag_names:
            raise ValueError(f"Generic tag '{generic_name}' does not exist")

        # Validate specific tag exists if provided
        if specific_name is not None and specific_name not in self.valid_tag_names:
            raise ValueError(f"Specific tag '{specific_name}' does not exist")

        self.mappings[calculated_name] = (generic_name, specific_name)

    def add_to_ignore(self, calculated_name):
        # Strip the prefix if present
        if calculated_name.startswith("Group Makeup Calculated: "):
            calculated_name = calculated_name[len("Group Makeup Calculated: ") :]
        self.ignored_tags.add(calculated_name)

    def get_generic_tag(self, calculated_name):
        if calculated_name.startswith("Group Makeup Calculated: "):
            calculated_name = calculated_name[len("Group Makeup Calculated: ") :]
        return self.mappings.get(calculated_name, (None, None))[0]

    def get_specific_tag(self, calculated_name):
        if calculated_name.startswith("Group Makeup Calculated: "):
            calculated_name = calculated_name[len("Group Makeup Calculated: ") :]
        return self.mappings.get(calculated_name, (None, None))[1]

    def is_ignored(self, calculated_name):
        if calculated_name.startswith("Group Makeup Calculated: "):
            calculated_name = calculated_name[len("Group Makeup Calculated: ") :]
        return calculated_name in self.ignored_tags


# Create the mapping instance with tag validation
all_tags = pl.DataFrame(stash.find_tags(fragment="id name"))
makeup_mapping = GroupMakeupMapping(all_tags)

# Add mappings for standard configurations (with both generic and specific tags)
makeup_mapping.add_mapping("1F", "Solo", "Solo Female")
makeup_mapping.add_mapping("1M", "Solo", "Solo Male")
makeup_mapping.add_mapping("1TF", "Solo", "Solo Trans")

makeup_mapping.add_mapping("1F1M", "Twosome", "Twosome (Straight)")
makeup_mapping.add_mapping("2F", "Twosome", "Twosome (Lesbian)")
makeup_mapping.add_mapping("1TF1F", "Twosome", "Twosome (Trans-Female)")
makeup_mapping.add_mapping("1TF1M", "Twosome", "Twosome (Trans-Male)")
makeup_mapping.add_mapping("2M", "Twosome", "Twosome (Gay)")
makeup_mapping.add_mapping("2TF", "Twosome", "Twosome (Trans)")
makeup_mapping.add_mapping("1TF1TM", "Twosome", "Twosome (Trans)")

# Add mappings for threesomes
makeup_mapping.add_mapping("2F1M", "Threesome", "Threesome (BGG)")
makeup_mapping.add_mapping("1F2M", "Threesome", "Threesome (BBG)")
makeup_mapping.add_mapping("3F", "Threesome", "Threesome (Lesbian)")
makeup_mapping.add_mapping("1TF2M", "Threesome", "Threesome (BBT)")
makeup_mapping.add_mapping("2TF1M", "Threesome", "Threesome (BTT)")
makeup_mapping.add_mapping("1TF1F1M", "Threesome", "Threesome (BGT)")
makeup_mapping.add_mapping("1TF2F", "Threesome", "Threesome (GGT)")
makeup_mapping.add_mapping("3TF", "Threesome", "Threesome (Trans)")
makeup_mapping.add_mapping("2TF1TM", "Threesome", "Threesome (Trans)")
makeup_mapping.add_mapping("2TF1F", "Threesome", "Threesome (GTT)")

# Add mappings for foursomes
makeup_mapping.add_mapping("2F2M", "Foursome", "Foursome (BBGG)")
makeup_mapping.add_mapping("3F1M", "Foursome", "Foursome (BGGG)")
makeup_mapping.add_mapping("1F3M", "Foursome", "Foursome (BBBG)")
makeup_mapping.add_mapping("4F", "Foursome", "Foursome (Lesbian)")
makeup_mapping.add_mapping("1TF1F2M", "Foursome", "Foursome (BBGT)")
makeup_mapping.add_mapping("1TF2F1M", "Foursome", "Foursome (BGGT)")
makeup_mapping.add_mapping("1TF3F", "Foursome", "Foursome (GGGT)")
makeup_mapping.add_mapping("1TF3M", "Foursome", "Foursome (BBBT)")
makeup_mapping.add_mapping("2TF2F", "Foursome", "Foursome (GGTT)")
makeup_mapping.add_mapping("2TF2M", "Foursome", "Foursome (BBTT)")
makeup_mapping.add_mapping("3TF1F", "Foursome", "Foursome (GTTT)")


# Add mappings for fivesomes
makeup_mapping.add_mapping("3F2M", "Fivesome", "Fivesome (BBBGG)")
makeup_mapping.add_mapping("2F3M", "Fivesome", "Fivesome (BBBGG)")
makeup_mapping.add_mapping("5F", "Fivesome", "Fivesome (Lesbian)")
makeup_mapping.add_mapping("1TF4M", "Fivesome", "Fivesome (GGGGT)")
makeup_mapping.add_mapping("1TF5M", "Fivesome", "Fivesome (BBBBT)")
makeup_mapping.add_mapping("2TF3F", "Fivesome", "Fivesome (GGGTT)")


# Sixsomes
makeup_mapping.add_mapping("3F3M", "Sixsome", "Sixsome (BBBGGG)")
makeup_mapping.add_mapping("6F", "Sixsome", "Sixsome (Lesbian)")
makeup_mapping.add_mapping("1F5M", "Sixsome", "Sixsome (BBBBBG)")
makeup_mapping.add_mapping("2F4M", "Sixsome", "Sixsome (BBBBGG)")
makeup_mapping.add_mapping("4F2M", "Sixsome", "Sixsome (BBGGGG)")


# Sevensomes
makeup_mapping.add_mapping("1TF6M", "Sevensome", "Sevensome (BBBBBBT)")
makeup_mapping.add_mapping("2F5M", "Sevensome", "Sevensome (BBBBBGG)")
makeup_mapping.add_mapping("3F4M", "Sevensome", "Sevensome (BBBBGGG)")
makeup_mapping.add_mapping("5F2M", "Sevensome", "Sevensome (BBGGGGG)")
makeup_mapping.add_mapping("4F3M", "Sevensome", "Sevensome (BBBGGGG)")


# Eightsomes
makeup_mapping.add_mapping("2F6M", "Eightsome", "Eightsome (BBBBBBGG)")
makeup_mapping.add_mapping("3F5M", "Eightsome", "Eightsome (BBBBBGGG)")
makeup_mapping.add_mapping("4F4M", "Eightsome", "Eightsome (BBBBGGGG)")
makeup_mapping.add_mapping("6F2M", "Eightsome", "Eightsome (BBGGGGGG)")
makeup_mapping.add_mapping("5F3M", "Eightsome", "Eightsome (BBBGGGGG)")


# Add orgy mappings
makeup_mapping.add_mapping("10F8M", "Orgy", "Orgy (Mixed)")
makeup_mapping.add_mapping("12F3M", "Orgy", "Orgy (Mixed)")
makeup_mapping.add_mapping("13F6M", "Orgy", "Orgy (Mixed)")
makeup_mapping.add_mapping("8F2M", "Orgy", "Orgy (Mixed)")
makeup_mapping.add_mapping("8F3M", "Orgy", "Orgy (Mixed)")
makeup_mapping.add_mapping("8F5M", "Orgy", "Orgy (Mixed)")
makeup_mapping.add_mapping("9F7M", "Orgy", "Orgy (Mixed)")
makeup_mapping.add_mapping("9F8M", "Orgy", "Orgy (Mixed)")

makeup_mapping.add_mapping("7F", "Orgy", "Orgy (Lesbian)")
makeup_mapping.add_mapping("8F", "Orgy", "Orgy (Lesbian)")
makeup_mapping.add_mapping("12F", "Orgy", "Orgy (Lesbian)")


# Add gangbang mappings (only generic tags)
makeup_mapping.add_mapping("1F4M", "Fivesome", "Fivesome (BBBBG)")
makeup_mapping.add_mapping("1F6M", "Sevensome", "Sevensome (BBBBBBG)")
makeup_mapping.add_mapping("1F7M", "Eightsome", "Eightsome (BBBBBBBG)")
makeup_mapping.add_mapping("1F8M", "Ninesome", "Ninesome (BBBBBBBBG)")
makeup_mapping.add_mapping("1F9M", "Gangbang")
makeup_mapping.add_mapping("1F10M", "Gangbang")
makeup_mapping.add_mapping("1F11M", "Gangbang")
makeup_mapping.add_mapping("1F12M", "Gangbang")
makeup_mapping.add_mapping("1F13M", "Gangbang")


# Add reverse gangbang mappings (only generic tags)
makeup_mapping.add_mapping("4F1M", "Reverse Gangbang")
makeup_mapping.add_mapping("5F1M", "Reverse Gangbang")
makeup_mapping.add_mapping("6F1M", "Reverse Gangbang")
makeup_mapping.add_mapping("7F1M", "Reverse Gangbang")
makeup_mapping.add_mapping("8F1M", "Reverse Gangbang")


# Get unique calculated tags to help with mapping
calculated_tags_df = pl.DataFrame(
    stash.find_tag("Group Makeup Calculated", fragment="id name children { id name }")[
        "children"
    ]
)

print("Calculated tags status:")
for tag in calculated_tags_df.sort("name").iter_rows(named=True):
    name = tag["name"].replace("Group Makeup Calculated: ", "")
    if makeup_mapping.is_ignored(name):
        # print(f"{name}: ignored")
        pass
    else:
        generic = makeup_mapping.get_generic_tag(name)
        specific = makeup_mapping.get_specific_tag(name)
        if generic is None:
            print(f"{name}: needs mapping")
        else:
            # print(f"{name}: {generic} -> {specific}")
            pass

Calculated tags status:
2F7M: needs mapping
39F: needs mapping
39F1M: needs mapping
3F6M: needs mapping
3F8M: needs mapping
3TF2M: needs mapping
4F8M: needs mapping
5F4M: needs mapping
6F12M: needs mapping
6F3M: needs mapping
6F4M: needs mapping
6TF: needs mapping
7F3M: needs mapping
7F4M: needs mapping
8F4M: needs mapping


In [26]:
new_tag_generic = "Sevensome"
new_tag_specific = "Sevensome (BBBGGGG)"
description = (
    "Exactly three male performers and four female performers, at the same time."
)


category_group_makeup = stash.find_tag("Category: Group Makeup")
print(category_group_makeup)

xsome_specific = stash.find_tag("Group Makeup Specific: " + new_tag_generic)
print(xsome_specific)

stash.create_tag(
    {
        "name": new_tag_specific,
        "description": description,
        "parent_ids": [category_group_makeup["id"], xsome_specific["id"]],
    }
)


eGRAPHQL_ERROR:['tagCreate'] tag with name 'Sevensome (BBBGGGG)' already exists


{'id': '7777', 'name': 'Category: Group Makeup', 'sort_name': '', 'description': 'StashDB category: Group Makeup', 'aliases': ['StashDB ID: c456b6a1-75c4-41d1-8d87-08d7e062c7d8'], 'ignore_auto_tag': False, 'created_at': '2024-09-19T06:44:42+03:00', 'updated_at': '2025-03-03T12:16:03+02:00', 'favorite': False, 'image_path': 'http://localhost:6969/tag/7777/image?t=1740996963&default=true', 'scene_count': 0, 'scene_marker_count': 0, 'image_count': 0, 'gallery_count': 0, 'performer_count': 0, 'studio_count': 0, 'group_count': 0, 'parents': [{'id': '7748'}], 'children': [{'id': '7428'}, {'id': '7431'}, {'id': '5712'}, {'id': '8556'}, {'id': '8857'}, {'id': '8251'}, {'id': '8721'}, {'id': '8722'}, {'id': '8744'}, {'id': '8725'}, {'id': '8723'}, {'id': '8555'}, {'id': '8558'}, {'id': '5899'}, {'id': '8270'}, {'id': '8540'}, {'id': '7819'}, {'id': '7885'}, {'id': '7896'}, {'id': '8539'}, {'id': '8426'}, {'id': '8427'}, {'id': '5937'}, {'id': '5938'}, {'id': '5939'}, {'id': '5940'}, {'id': '594

In [27]:
def check_scene_tags(scene, makeup_mapping, generic_tag_ids, specific_tag_ids):
    """Check if a scene has the correct tags according to the mapping"""

    # Get the calculated tag if it exists
    calculated_tag = next(
        (
            tag["name"]
            for tag in scene["tags"]
            if tag["name"].startswith("Group Makeup Calculated: ")
        ),
        None,
    )

    if not calculated_tag:
        return None  # Skip scenes without calculated tags

    # Get expected generic and specific tags
    expected_generic = makeup_mapping.get_generic_tag(calculated_tag)
    expected_specific = makeup_mapping.get_specific_tag(calculated_tag)

    # Find actual generic and specific tags
    actual_generic_tags = [
        tag["name"] for tag in scene["tags"] if tag["id"] in generic_tag_ids
    ]
    actual_specific_tags = [
        tag["name"] for tag in scene["tags"] if tag["id"] in specific_tag_ids
    ]

    # Check if tags match expectations
    if makeup_mapping.is_ignored(calculated_tag):
        return None

    issues = []

    # If we expect both generic and specific
    if expected_specific:
        if len(actual_generic_tags) != 1 or actual_generic_tags[0] != expected_generic:
            issues.append(
                f"Generic tag mismatch - Expected: {expected_generic}, Got: {actual_generic_tags}"
            )
        if (
            len(actual_specific_tags) != 1
            or actual_specific_tags[0] != expected_specific
        ):
            issues.append(
                f"Specific tag mismatch - Expected: {expected_specific}, Got: {actual_specific_tags}"
            )

    # If we only expect generic
    else:
        if len(actual_generic_tags) != 1 or actual_generic_tags[0] != expected_generic:
            issues.append(
                f"Generic tag mismatch - Expected: {expected_generic}, Got: {actual_generic_tags}"
            )
        if len(actual_specific_tags) > 0:
            issues.append(f"Unexpected specific tags present: {actual_specific_tags}")

    if issues:
        return {
            "id": scene["id"],
            "title": scene["title"],
            "calculated_tag": calculated_tag,
            "expected_generic": expected_generic,
            "expected_specific": expected_specific,
            "actual_generic": actual_generic_tags,
            "actual_specific": actual_specific_tags,
            "issues": issues,
        }

    return None

In [28]:
# Get all scenes
all_scenes = pl.DataFrame(stash.find_scenes(fragment="id title tags { id name }"))

# Check each scene
issues = []
for scene in all_scenes.iter_rows(named=True):
    result = check_scene_tags(
        scene,
        makeup_mapping,
        group_makeup_generic_tag_ids,
        group_makeup_specific_tag_ids,
    )
    if result:
        issues.append(result)

# Convert results to DataFrame for better visualization
if issues:
    issues_df = pl.DataFrame(issues)
    print(f"Found {len(issues)} scenes with incorrect tags:")
    print(issues_df)
else:
    print("All scenes have correct tags according to mappings!")

# Optional: Show detailed breakdown of issues
for issue in issues:
    print(f"\nScene {issue['id']} - {issue['title']}")
    print(f"Calculated tag: {issue['calculated_tag']}")
    for problem in issue["issues"]:
        print(f"- {problem}")


Found 4368 scenes with incorrect tags:
shape: (4_368, 8)
┌───────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┐
│ id    ┆ title      ┆ calculated ┆ expected_g ┆ expected_s ┆ actual_gen ┆ actual_spe ┆ issues     │
│ ---   ┆ ---        ┆ _tag       ┆ eneric     ┆ pecific    ┆ eric       ┆ cific      ┆ ---        │
│ str   ┆ str        ┆ ---        ┆ ---        ┆ ---        ┆ ---        ┆ ---        ┆ list[str]  │
│       ┆            ┆ str        ┆ str        ┆ str        ┆ list[str]  ┆ list[str]  ┆            │
╞═══════╪════════════╪════════════╪════════════╪════════════╪════════════╪════════════╪════════════╡
│ 3438  ┆ Eating Ass ┆ Group      ┆ Solo       ┆ Solo       ┆ ["Twosome" ┆ ["Twosome  ┆ ["Generic  │
│       ┆ & Sucking  ┆ Makeup Cal ┆            ┆ Female     ┆ ]          ┆ (Straight) ┆ tag        │
│       ┆ Cock at S… ┆ culated:   ┆            ┆            ┆            ┆ "]         ┆ mismatch - │
│       ┆            ┆ 1F         

In [29]:

# Start Chrome with remote debugging (run this in PowerShell before running the notebook)
# Start-Process "chrome.exe" -ArgumentList "--remote-debugging-port=9222", "--user-data-dir=C:\temp\chrome-debug"

import requests
import json
from libraries import browser

def check_browser_connection():
    """Check if Chrome is running with remote debugging enabled"""
    try:
        response = requests.get('http://localhost:9222/json', timeout=5)
        if response.status_code == 200:
            pages = json.loads(response.text)
            print(f"✅ Chrome is running with remote debugging")
            print(f"📊 Found {len(pages)} open tabs/pages")
            return True
        else:
            print(f"❌ Chrome debugging port responded with status {response.status_code}")
            return False
    except requests.exceptions.ConnectionError:
        print("❌ Cannot connect to Chrome debugging port")
        print("🔧 Please start Chrome with: --remote-debugging-port=9222")
        return False
    except Exception as e:
        print(f"❌ Error checking browser connection: {str(e)}")
        return False

# Test the browser connection
browser_ready = check_browser_connection()


✅ Chrome is running with remote debugging
📊 Found 9 open tabs/pages


In [30]:
# Check if browser is ready before proceeding
if not browser_ready:
    print("⚠️  Browser is not ready. Please start Chrome with remote debugging first.")
    print("Run this command in PowerShell:")
    print('Start-Process "chrome.exe" -ArgumentList "--remote-debugging-port=9222", "--user-data-dir=C:\\temp\\chrome-debug"')
    print("\nThen re-run the browser connection check cell above.")
else:
    print("🚀 Browser is ready! Opening tabs...")
    
    issues_batch = issues_df.head(10)
    
    # Create list of full URLs
    urls = [
        f"https://stash.chiefsclub.com/scenes/{issue['id']}"
        for issue in issues_batch.iter_rows(named=True)
    ]
    
    print(f"📋 Opening {len(urls)} URLs:")
    for i, url in enumerate(urls, 1):
        print(f"  {i}. {url}")
    
    # Open all URLs at once
    results = browser.open_or_update_tabs(urls)
    
    # Check results
    successful = sum(1 for success in results.values() if success)
    failed = len(results) - successful
    
    print(f"\n✅ Successfully opened: {successful}/{len(urls)} tabs")
    if failed > 0:
        print(f"❌ Failed to open: {failed} tabs")
        for url, success in results.items():
            if not success:
                print(f"  - {url}")


🚀 Browser is ready! Opening tabs...
📋 Opening 10 URLs:
  1. https://stash.chiefsclub.com/scenes/3438
  2. https://stash.chiefsclub.com/scenes/26288
  3. https://stash.chiefsclub.com/scenes/26934
  4. https://stash.chiefsclub.com/scenes/28038
  5. https://stash.chiefsclub.com/scenes/31762
  6. https://stash.chiefsclub.com/scenes/32113
  7. https://stash.chiefsclub.com/scenes/1849
  8. https://stash.chiefsclub.com/scenes/5510
  9. https://stash.chiefsclub.com/scenes/5755
  10. https://stash.chiefsclub.com/scenes/5756
Got response from Chrome: 200
Found 1 existing pages
Creating new tab for https://stash.chiefsclub.com/scenes/3438
Found matching tab for https://stash.chiefsclub.com/scenes/26288, bringing to front
Creating new tab for https://stash.chiefsclub.com/scenes/26934
Creating new tab for https://stash.chiefsclub.com/scenes/28038
Creating new tab for https://stash.chiefsclub.com/scenes/31762
Creating new tab for https://stash.chiefsclub.com/scenes/32113
Creating new tab for https:/

In [12]:
group_makeup_verified_tag_id = stash.find_tag("Group Makeup Verified")["id"]
group_makeup_calculated_parent_tag = stash.find_tag(
    "Group Makeup Calculated", fragment="id name children { id name }"
)
group_makeup_calculated_tag_ids = [
    tag["id"] for tag in group_makeup_calculated_parent_tag["children"]
]

In [ ]:
unverified_scenes = stash.find_scenes(
    {
        "tags": {
            "value": [],
            "modifier": "INCLUDES",
            "excludes": [group_makeup_verified_tag_id],
        }
    },
    fragment="id title date tags { id name }",
)

# Convert to polars DataFrame and explode tags
unverified_df = pl.DataFrame(unverified_scenes).explode("tags")

# Filter for only group makeup calculated tags
group_makeup_df = (
    unverified_df.filter(
        pl.col("tags").struct.field("id").is_in(group_makeup_calculated_tag_ids)
    )
    .group_by("tags")
    .agg(pl.col("id").count().alias("scene_count"))
    .sort("scene_count", descending=True)
)
print(group_makeup_df.sum().select(pl.col("scene_count")))
group_makeup_df
